## 脚本编程与系统管理

首先补充一些基本的在python脚本中执行系统命令的代码。cank 

In [53]:
import os
os.system('ls')

1

In [54]:
tmp = os.popen('ls *.py').readlines()
tmp

[]

In [55]:
import subprocess

p = subprocess.Popen('ls', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line)
retval = p.wait()

b"'ls' \xb2\xbb\xca\xc7\xc4\xda\xb2\xbf\xbb\xf2\xcd\xe2\xb2\xbf\xc3\xfc\xc1\xee\xa3\xac\xd2\xb2\xb2\xbb\xca\xc7\xbf\xc9\xd4\xcb\xd0\xd0\xb5\xc4\xb3\xcc\xd0\xf2\r\n"
b'\xbb\xf2\xc5\xfa\xb4\xa6\xc0\xed\xce\xc4\xbc\xfe\xa1\xa3\r\n'


### 命令行解析器

Python 命令行与参数解析方法有很多工具，这里学习使用python 自带的argparse ，来说明python 如何进行命令行解析。主要参考了

- [HelloGitHub-Article](https://github.com/HelloGitHub-Team/Article)
- [argparse模块用法实例详解](https://zhuanlan.zhihu.com/p/56922793)
- python[官方文档](https://docs.python.org/zh-cn/3/library/argparse.html)
- [Python-argparse-命令行与参数解析](https://zhuanlan.zhihu.com/p/34395749)

通俗来说，命令行与参数解析就是当你输入cmd 打开dos 交互界面时候，启动程序要进行的参数给定。比如在dos 界面输入：

```code
python openPythonFile.py "a" -b "number"
```

其中，"a" -b等就是命令行与参数解析要做的事情。先不用深究参数的含义，这里就是个示例，简而言之，就是设计程序在**运行时必须给定某些额外参数**才能运行，也就是如果设置了命令行参数解析，那么各种编译器按F5 是无法直接运行程序的。这样的目的之一是不能随便就能运行脚本，可以达到一定程度上的安全功能。

那肯定就会好奇命令行中敲入一段命令后，是如何被解析执行的？自己如何实现一个命令行工具来帮助执行和处理任务？如何利用python库来帮助实现？

这一节就主要记录如何使用Python 内置的 argparse 标准库解析命令行。

argparse 作为 Python 内置的标准库，提供了较为简单的方式来编写命令行接口。当你在程序中定义需要哪些参数，argparse 便会从 sys.argv 中获取命令行输入进行解析，对正确或非法输入做出响应，也可以自动生成帮助信息和使用说明。

总体上分为三大步：

- 创建解析：设置解析器，后续对命令行的解析就**依赖于这个解析器**，它能够**将命令行字符串转换为Python对象**。通过实例化 argparse.ArgumentParser，给定一些选填参数，就可以设置一个解析器
- 添加参数：通过ArgumentParser.add_argument 方法来**为解析器设置参数信息**，以告诉解析器命令行字符串中的**哪些内容**应解析为**哪些类型的Python对象**。注意，每一个参数都要单独设置，需要两个参数就用两个add_argument
- 解析参数：定义好参数后，就可以使用 ArgumenteParser.**parse_args 方法来解析一组命令行参数字符串**了。默认情况下，参数取自**sys.argv[1:]**,它就是我们在命令行敲入的**一段命令（不含文件名）所对应的一个字符串列表**，比如，若输入 python3 cmd.py --sum 1 2 3，那么sys.argsv[1:]就是['--sum','1','2','3']。

基本的业务逻辑是这样的。解析好命令行后，我们就可以从解析结果中获取每个参数的值，进而根据自己的业务需求做进一步的处理。比如，对于上文中所定义的nums参数，我们可以通过解析后的结果中的accumulate方法对其进行求最大值或求和（取决于是否提供 --sum 参数）。

下面就给出一个较完整的代码示例。

In [56]:
import argparse

# 1. 设置解析器
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)

# 2. 定义参数
# 添加 nums 参数，在使用信息中显示为 num
# 其类型为 int，且支持输入多个，且至少需要提供一个
parser.add_argument('nums',  metavar='num', type=int, nargs='+',
                    help='a num for the accumulator')
# 添加 --sum 参数，该参数被 parser 解析后所对应的属性名为 accumulate
# 若不提供 --sum，默认值为 max 函数，否则为 sum 函数
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the nums (default: find the max)')

# 3. 解析命令行
args = parser.parse_args(['--sum', '-1', '0', '1'])
print(args) # 结果：Namespace(accumulate=<built-in function sum>, nums=[-1, 0, 1])

# 4. 业务逻辑
result = args.accumulate(args.nums)
print(result)

Namespace(nums=[-1, 0, 1], accumulate=<built-in function sum>)
0


若我们需要对一组数字求和，只需执行：

```Shell
$ python3 cmd.py --sum -1 0 1
0
```

若需要对一组数字求最大值，只需执行：

```Shell
$ python3 cmd.py -1 0 1
1
```

如果给定的参数不是数字，则会报错提示：

```Shell
$ python3 cmd.py a b c
usage: cmd.py [-h] [--sum] num [num ...]
cmd.py: error: argument num: invalid int value: 'a'
```

我们还可以通过 -h 或 --help 参数查看其自动生成的使用说明和帮助：

```Shell
usage: cmd.py [-h] [--sum] num [num ...]

My Cmd Line Program

positional arguments:
  num         a num for the accumulator

optional arguments:
  -h, --help  show this help message and exit
  --sum       sum the nums (default: find the max)
```

接下来进一步探讨关于argparse更多复杂的情况，比如各种类型参数、参数前缀、参数组、互斥选项、嵌套解析、自定义帮助等等。主要要认识的问题是：argparse支持哪些类型的参数？这些参数该如何配置？

In [57]:
import argparse

# 1. 设置解析器
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)

1. 参数动作

In [58]:
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the nums (default: find the max)')

_StoreConstAction(option_strings=['--sum'], dest='accumulate', nargs=0, const=<built-in function sum>, default=<built-in function max>, type=None, choices=None, help='sum the nums (default: find the max)', metavar=None)

这里面的 action，也就是 参数动作，究竟是用来做什么的呢？

想象一下，当我们在命令行输入**一串参数**后，对于**不同类型的参数是希望做不同的处理**的。 那么 **参数动作** 其实就是告诉解析器，我们希望**对应的参数该被如何处理**。比如，参数值是该被存成一个值呢，还是追加到一个列表中？是当成布尔的 True 呢，还是 False？

参数动作 被分成了如下 8 个类别：

- store —— 保存参数的值，这是**默认**的参数动作。它通常用于给一个参数指定值，如指定名字：

In [59]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--name')
parser.parse_args(['--name', 'Eric'])

Namespace(name='Eric')

- store_const —— 保存被 const 命名的固定值。当我们想通过**是否给定参数**来起到**标志**的作用，给定就取某个值，就可以使用该参数动作，如：

In [60]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--sum', action='store_const', const=sum)
parser.parse_args(['--sum'])

Namespace(sum=<built-in function sum>)

- store_true 和 store_false —— 是 store_const 的特殊情况，用来分别保存 True 和 False。如果为指定参数，则其默认值分别为 False 和 True，如：

In [61]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--use', action='store_true')
parser.add_argument('--nouse', action='store_false')
parser.parse_args(['--use', '--nouse'])

Namespace(use=True, nouse=False)

In [62]:
parser.parse_args([])

Namespace(use=False, nouse=True)

- append —— 将参数值追加保存到一个列表中。它常常用于命令行中允许多个相同选项，如：

In [63]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--file', action='append')
parser.parse_args(['--file', 'f1', '--file', 'f2'])

Namespace(file=['f1', 'f2'])

- append_const —— 将 const 命名的固定值追加保存到一个列表中（const 的默认值为 None）。它常常用于将多个参数所对应的固定值都保存在同一个列表中，相应的需要 dest 入参来配合，以放在同一个列表中，如：

不指定 dest 入参，则固定值保存在以参数名命名的变量中

In [64]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--int', action='append_const', const=int)
parser.add_argument('--str', action='append_const', const=str)
parser.parse_args(['--int', '--str'])

Namespace(int=[<class 'int'>], str=[<class 'str'>])

指定 dest 入参，则固定值保存在 dest 命名的变量中

In [65]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--int', dest='types', action='append_const', const=int)
parser.add_argument('--str', dest='types', action='append_const', const=str)
parser.parse_args(['--int', '--str'])

Namespace(types=[<class 'int'>, <class 'str'>])

- count —— 计算参数出现次数，如：

In [66]:
parser = argparse.ArgumentParser(
    description='My Cmd Line Program',
)
parser.add_argument('--increase', '-i', action='count')
parser.parse_args(['--increase', '--increase'])

Namespace(increase=2)

In [67]:
parser.parse_args(['-iii'])

Namespace(increase=3)

- help —— 打印解析器中所有选项和参数的完整帮助信息，然后退出。

- version —— 打印命令行版本，通过指定 version 入参来指定版本，调用后退出。如：

In [68]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--version', action='version', version='%(prog)s 1.0')
parser.parse_args(['--version'])

CMD 1.0


SystemExit: 0

C:\Users\hust2\miniconda3\envs\hydrus\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


2. 参数类别

如果说 参数动作 定义了**解析器在接收到参数后该如何处理参数**，那么 参数类别 就是告诉解析器**这个参数的元信息**，也就是参数是什么样的。比如，参数是字符串呢？还是布尔类型呢？参数是在几个值中可选的呢？还是可以给定值，等等。

可选参数 顾名思义就是参数是可以加上，或不加上。**默认**情况下，通过 ArgumentParser.add_argument 添加的参数就是**可选参数**。

可以通过 - 来指定**短参数**，也就是名称短的参数；也可以通过 -- 来指定**长参数**，也就是名称长的参数。当然也可以两个都指定。

可选参数通常用于：用户提供一个参数以及对应值，则使用该值；若不提供，则使用默认值。如：

In [69]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--name', '-n')
parser.parse_args(['--name', 'Eric'])  # 通过长参数指定名称

Namespace(name='Eric')

In [70]:
parser.parse_args(['-n', 'Eric']) # 通过短参数指定名称

Namespace(name='Eric')

In [71]:
parser.parse_args([]) # 不指定则默认为 None

Namespace(name=None)

参数类型 就是解析器**参数值是要作为什么类型去解析**，默认情况下是 str 类型。我们可以通过 type 入参来指定参数类型。

argparse 所支持的参数类型多种多样，可以是 int、float、bool等，比如：

In [72]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('-i', type=int)
parser.add_argument('-f', type=float)
parser.add_argument('-b', type=bool)
parser.parse_args(['-i', '1', '-f', '2.1', '-b', '0'])

Namespace(i=1, f=2.1, b=True)

更厉害的是，type 入参还可以是**可调用(callable)对象**。这就给了我们很大的想象空间，可以指定 type=open 来把参数值作为文件进行处理，也可以指定自定义函数来进行类型检查和类型转换。

作为文件进行处理：

In [73]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--file', type=open)
parser.parse_args(['--file', 'test.txt'])

Namespace(file=<_io.TextIOWrapper name='test.txt' mode='r' encoding='cp936'>)

使用自定义函数进行处理，入参为参数值，需返回转换后的结果。 比如，对于参数 --num，我们希望当其值小于 1 时则返回 1，大于 10 时则返回 10：

In [74]:
def limit(string):
    num = int(string)
    if num < 1:
        return 1
    if num > 10:
        return 10
    return num
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--num', type=limit)
parser.parse_args(['--num', '-1'])  # num 小于1，则取1

Namespace(num=1)

In [75]:
parser.parse_args(['--num', '15'])  # num 大于10，则取10

Namespace(num=10)

In [76]:
parser.parse_args(['--num', '5'])  # num 在1和10之间，则取原来的值

Namespace(num=5)

3. 参数默认值

参数默认值 用于在命令行中不传参数值的情况下的默认取值，可通过 default 来指定。如果不指定该值，则参数默认值为 None。

比如：

In [77]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('-i', default=0, type=int)
parser.add_argument('-f', default=3.14, type=float)
parser.add_argument('-b', default=True, type=bool)
parser.parse_args([])

Namespace(i=0, f=3.14, b=True)

4. 位置参数

位置参数 就是通过位置而非是 - 或 -- 开头的参数来指定参数值。

比如，我们可以指定两个位置参数 x 和 y ，先添加的 x 位于第一个位置，后加入的 y 位于第二个位置。那么在命令行中输入 1 2的时候，分别对应到的就是 x 和 y：

In [78]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('x')
parser.add_argument('y')
parser.parse_args(['1', '2'])

Namespace(x='1', y='2')

5. 可选值

可选值 就是**限定参数值的内容**，通过 choices 入参指定。

有些情况下，我们可能需要限制用户输入参数的内容，只能在预设的几个值中选一个，那么 可选值 就派上了用场。

比如，指定文件读取方式限制为 read-only 和 read-write：

In [79]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--mode', choices=('read-only', 'read-write'))
parser.parse_args(['--mode', 'read-only'])

Namespace(mode='read-only')

In [80]:
parser.parse_args(['--mode', 'read'])

usage: CMD [-h] [--mode {read-only,read-write}]
CMD: error: argument --mode: invalid choice: 'read' (choose from 'read-only', 'read-write')
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1851, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2060, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2000, in consume_optional
    take_action(action, args, option_string)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1912, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2444, in _get_values
    self._check_value(action, value)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2500, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument --mode: invalid choice: 'read' (choose from 'read-only', 'read-write')

Du

TypeError: object of type 'NoneType' has no len()

6. 互斥参数

互斥参数 就是多个参数之间彼此互斥，不能同时出现。使用互斥参数首先通过 ArgumentParser.add_mutually_exclusive_group 在解析器中添加一个互斥组，然后在这个组里添加参数，那么组内的所有参数都是互斥的。

比如，我们希望通过命令行来告知乘坐的交通工具，要么是汽车，要么是公交，要么是自行车，那么就可以这么写：

In [81]:
parser = argparse.ArgumentParser(prog='CMD')
group = parser.add_mutually_exclusive_group()
group.add_argument('--car', action='store_true')
group.add_argument('--bus', action='store_true')
group.add_argument('--bike', action='store_true')
parser.parse_args([])  # 什么都不乘坐

Namespace(car=False, bus=False, bike=False)

In [82]:
parser.parse_args(['--bus'])  # 乘坐公交

Namespace(car=False, bus=True, bike=False)

In [83]:
parser.parse_args(['--bike'])  # 骑自行车

Namespace(car=False, bus=False, bike=True)

In [84]:
parser.parse_args(['--bike', '--car'])  # 又想骑车，又想坐车，那是不行的

usage: CMD [-h] [--car | --bus | --bike]
CMD: error: argument --car: not allowed with argument --bike
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1851, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2060, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 2000, in consume_optional
    take_action(action, args, option_string)
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\argparse.py", line 1923, in take_action
    raise ArgumentError(action, msg % action_name)
argparse.ArgumentError: argument --car: not allowed with argument --bike

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hust2\miniconda3\envs\hydrus\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hust2\AppDat

TypeError: object of type 'NoneType' has no len()

7. 可变参数列表

可变参数列表 用来定义一个参数可以有多个值，且能通过 nargs 来定义值的个数。

若 nargs=N，N为一个数字，则要求该参数提供 N 个值，如：

In [85]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs=2)
print(parser.parse_args(['--foo', 'a', 'b']))

Namespace(foo=['a', 'b'])


若 nargs=?，则要求改参数提供 0 或 1 个值，如：

In [86]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs='?')
parser.parse_args(['--foo'])

Namespace(foo=None)

In [87]:
parser.parse_args(['--foo', 'a'])

Namespace(foo='a')

若 nargs=*，则要求改参数提供 0 或多个值，如：

In [88]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs='*')
parser.parse_args(['--foo', 'a', 'b', 'c', 'd', 'e'])

Namespace(foo=['a', 'b', 'c', 'd', 'e'])

若 nargs=+，则要求改参数至少提供 1 个值，如：

In [89]:
parser = argparse.ArgumentParser(prog='CMD')
parser.add_argument('--foo', nargs='+')
parser.parse_args(['--foo', 'a'])

Namespace(foo=['a'])

小结下。

add_argument 方法定义**单个**的命令行参数应当**如何解析**。每个形参更多的描述：

- name or flags - 一个命名或者一个选项字符串的列表，例如 foo 或 -f, --foo。
- action - 当参数在命令行中出现时使用的动作基本类型。
- nargs - 命令行参数应当消耗的数目。
- const - 被一些 action 和 nargs 选择所需求的常数。
- default - 当参数未在命令行中出现时使用的值。
- type - 命令行参数应当被转换成的类型。
- choices - 可用的参数的容器。
- required - 此命令行选项是否可省略 （仅选项可用）。
- help - 一个此选项作用的简单描述。
- metavar - 在使用方法消息中使用的参数值示例。
- dest - 解析后的参数名称，默认情况下，对于可选参数选取最长的名称，中划线转换为下划线. 

然后一个比较完整的，需要在命令行中执行的例子如下，对应的python文件是argv_argparse.py.

调用方式：

```Shell
python argv_argparse.py -h
python argv_argparse.py xiaoming 1991.11.11
python argv_argparse.py xiaoming 1991.11.11 -p xiaohong xiaohei -a 25 -r han -s female -o 1 2 3 4 5 6
```

-h表示调出help信息。

以上是参数动作和参数类别相关内容，接下来继续深入了解 argparse 的功能，包括如何修改参数前缀，如何定义参数组，如何定义嵌套的解析器，如何编写自定义动作等。

1. 帮助

自动生成帮助

当你在命令行程序中指定 -h 或 --help 参数时，都会输出帮助信息。而 argparse 可通过指定 add_help 入参为 True 或不指定，以达到自动输出帮助信息的目的。

In [90]:
import argparse
parser = argparse.ArgumentParser(add_help=True)
parser.add_argument('--foo')
parser.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [--foo FOO]

optional arguments:
  -h, --help  show this help message and exit
  --foo FOO


SystemExit: 0

自定义帮助

ArgumentParser 使用 formatter_class 入参来控制所输出的帮助格式。 比如，通过指定 formatter_class=argparse.RawTextHelpFormatter，我们可以让帮助内容遵循原始格式：

In [91]:
import argparse
parser = argparse.ArgumentParser(
    add_help=True,
    formatter_class=argparse.RawTextHelpFormatter,
    description="""description  raw formatted"""
)
parser.add_argument(
    '-a', action="store_true",
    help="""argument raw formatted"""
)
parser.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [-a]

description  raw formatted

optional arguments:
  -h, --help  show this help message and exit
  -a          argument raw formatted


SystemExit: 0

2. 参数组

有时候，我们需要给参数分组，以使得在显示帮助信息时能够显示到一起。

比如某命令行支持三个参数选项 --user、--password和--push，前两者需要放在一个名为 authentication 的分组中以表示它们是身份认证信息。那么我们可以用 ArgumentParser.add_argument_group 来满足：

In [92]:
import argparse
parser = argparse.ArgumentParser()
group = parser.add_argument_group('authentication')
group.add_argument('--user', action="store")
group.add_argument('--password', action="store")
parser.add_argument('--push', action='store')
parser.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [--user USER] [--password PASSWORD] [--push PUSH]

optional arguments:
  -h, --help           show this help message and exit
  --push PUSH

authentication:
  --user USER
  --password PASSWORD


SystemExit: 0

3. 选项参数前缀

不知你是否注意到，在不同平台上命令行程序的选项参数前缀可能是不同的。比如在 Unix 上，其前缀是 -；而在 Windows 上，大多数命令行程序（比如 findstr）的选项参数前缀是 /。

在 argparse 中，选项参数前缀默认采用 Unix 命令行约定，也就是 -。但它也支持自定义前缀，下面是一个例子：

In [93]:
import argparse
parser = argparse.ArgumentParser(
    description='Option prefix',
    prefix_chars='-+/',
)

parser.add_argument('-power', action="store_false",
                    default=None,
                    help='Set power off',
                   )
parser.add_argument('+power', action="store_true",
                    default=None,
                    help='Set power on',
                   )
parser.add_argument('/win',
                    action="store_true",
                    default=False)
parser.parse_args(['-power'])

Namespace(power=False, win=False)

In [94]:
parser.parse_args(['+power', '/win'])

Namespace(power=True, win=True)

在这个例子中，我们指定了三个选项参数前缀 -、+和/，从而：

- 通过指定选项参数 -power，使得 power=False
- 通过指定选项参数 +power，使得 power=True
- 通过指定选项参数 /win，使得 win=True

### 读取配置文件

很多情况下，我们需要通过配置文件来定义一些参数性质的数据，因为配置文件作为一种可读性很好的格式，非常适用于存储程序中的配置数据。 

在每个配置文件中，配置数据会被分组（比如例子中的“installation”、 “debug” 和 “server”）。 每个分组在其中指定对应的各个变量值。那么如何读取普通.ini格式的配置文件？

在python中，configparser 模块能被用来读取配置文件。例如，假设有配置文件config.ini。下面给出读取代码：

In [95]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read('config.ini')

['config.ini']

In [96]:
cfg.sections()

['installation', 'debug', 'server']

In [97]:
cfg.get('installation','library')

'/usr/local/lib'

In [98]:
cfg.getboolean('debug','log_errors')

True

In [99]:
cfg.getint('server','port')

8080

In [100]:
cfg.getint('server','nworkers')

32

In [101]:
print(cfg.get('server','signature'))


Brought to you by the Python Cookbook


还可以读取一个section下的所有keys或所有键值对，参考：[Python 读取写入配置文件 —— ConfigParser](https://blog.csdn.net/jiede1/article/details/79064780)

In [102]:
cfg.options("installation") 

['library', 'include', 'bin', 'prefix']

In [103]:
cfg.items("installation")  

[('library', '/usr/local/lib'),
 ('include', '/usr/local/include'),
 ('bin', '/usr/local/bin'),
 ('prefix', '/usr/local')]

如果需要，还能修改配置并使用 cfg.write() 方法将其写回到文件中。例如：

In [104]:
cfg.set('server','port','9000')
cfg.set('debug','log_errors','False')
import sys
cfg.write(sys.stdout)

[installation]
library = %(prefix)s/lib
include = %(prefix)s/include
bin = %(prefix)s/bin
prefix = /usr/local

[debug]
log_errors = False
show_warnings = False

[server]
port = 9000
nworkers = 32
pid-file = /tmp/spam.pid
root = /www/root
signature = 
	Brought to you by the Python Cookbook



### 日志功能

即在脚本和程序中将诊断信息写入日志文件。打印日志最简单方式是使用 logging 模块。

这小节的内容主要参考了：

- [python必掌握模块(四）logging模块用法](https://zhuanlan.zhihu.com/p/56968001)
- [Python模块学习之Logging日志模块](https://y4er.com/post/python-logging/)

日志是 学习任何编程语言都有必要掌握的核心模块。因为当把python代码放入到**生产环境**中的时候，我们只能看到代码运行的结果，我们不知道的是代码每一步过程的最终运行状态。

如果代码中间过程出现了问题的话，logging库的引用得出的日志记录可以帮助我们排查程序运行错误步骤的。方便我们修复代码，快速排查问题。

logging模块是Python内置的标准模块，主要用于输出运行日志，可以设置输出日志的等级、日志保存路径、日志文件回滚等；相比print，具备如下优点：

- 可以通过设置不同的日志等级，在release版本中只输出重要信息，而不必显示大量的调试信息：print将所有信息都输出到标准输出中，严重影响开发者从标准输出中查看其它数据；logging则可以由开发者决定将信息输出到什么地方，以及怎么输出
- logging具有更灵活的格式化功能，比如运行时间、模块信息
- print输出都在控制台上，logging可以输出到任何位置，比如文件甚至是远程服务器

logging 的模块结构如下：

- Logger	记录日志时创建的对象，调用其方法来传入日志模板和信息生成日志记录
- Log Record	Logger对象生成的一条条记录
- Handler	处理日志记录，输出或者存储日志记录
- Formatter	格式化日志记录
- Filter	日志过滤器
- Parent Handler	Handler之间存在分层关系

In [105]:
import logging
import sys
logger = logging.getLogger("Your Logger")
logger.setLevel(logging.DEBUG)
# 标准输出流，输出到控制台，用sys.stdout的话，就是输出白色的
# handler = logging.StreamHandler(sys.stdout)
handler = logging.StreamHandler()
formatter = logging.Formatter(fmt='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y/%m/%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.info("this is info msg")
logger.debug("this is debug msg")
logger.warning("this is warn msg")
logger.error("this is error msg")

2021/07/29 17:58:42 - Your Logger - INFO - this is info msg
INFO:Your Logger:this is info msg
2021/07/29 17:58:42 - Your Logger - DEBUG - this is debug msg
DEBUG:Your Logger:this is debug msg
2021/07/29 17:58:42 - Your Logger - WARNING - this is warn msg
2021/07/29 17:58:42 - Your Logger - ERROR - this is error msg
ERROR:Your Logger:this is error msg


理解下这个例子。首先，创建了一个logger对象，来作为生成日志记录的对象，然后设置输出级别（所有级别低于此级别的日志消息都会被忽略掉）。 

接着创建了一个 StramHandler对象来处理日志。

随后创建一个formatter对象来格式化输出日志记录。构造最终的日志消息的时候我们使用了%操作符来格式化消息字符串。

然后把formatter 赋给 handler。

最后handler处理器添加到logger对象，完成整个处理流程。

下面稍作补充解释。

前面说到，设置输出级别时候，低于此级别的消息就被忽略了，低于是怎么判断的？其实logging 的 这些 level 常数是对应特定的整数值的，所以设置logging level的时候，也可以直接使用对应的整数来赋值。

In [106]:
print("logging.DEBUG:",logging.DEBUG)
print("logging.INFO:",logging.INFO)
print("logging.WARNING:",logging.WARNING)
print("logging.ERROR:",logging.ERROR)
print("logging.CRITICAL:",logging.CRITICAL)

logging.DEBUG: 10
logging.INFO: 20
logging.ERROR: 40
logging.CRITICAL: 50


从值上就可以看出来 DEBUG最低，然后依次往上，级别越来越高。

logging 提供的Handler有很多，比如：

- StreamHandler	logging.StreamHandler	日志输出到流，可以是 sys.stderr，sys.stdout 或者文件
- FileHandler	logging.FileHandler	日志输出到文件
- SMTPHandler	logging.handlers.SMTPHandler	远程输出日志到邮件地址
- SysLogHandler	logging.handlers.SysLogHandler	日志输出到syslog
- HTTPHandler	logging.handlers.HTTPHandler	通过”GET”或者”POST”远程输出到HTTP服务器

如果需要设置一个全局的logger以供使用，可以参考：
- https://blog.csdn.net/weixin_42526352/article/details/90242840
- https://blog.csdn.net/brucewong0516/article/details/82817008

这里也给出例子--globalLog.py。

In [107]:
from globalLog import hydro_logger

hydro_logger.info("this is info msg")
hydro_logger.debug("this is debug msg")
hydro_logger.warning("this is warn msg")
hydro_logger.error("this is error msg")

this is warn msg
this is error msg
ERROR:globalLog:this is error msg


## 测试、调试与异常

在Python测试代码之前没有编译器来分析代码，因此使得测试成为开发的一个重要部分。这里记录一些关于测试、调试和异常处理的常见问题。

### 关于单元测试

之前basic部分已经记录了一些unittest内容，这里补充一些诸如mock等概念的基本内容。主要参考了：[Python Mocking, You Are A Tricksy Beast](https://medium.com/python-pandemonium/python-mocking-you-are-a-tricksy-beast-6c4a1f8d19b2)，[An Introduction to Mocking in Python](https://www.toptal.com/python/an-introduction-to-mocking-in-python)和[Understanding the Python Mock Object Library](https://realpython.com/python-mock-library/#patch-as-a-decorator)。

#### 为什么要使用mock

测试是验证逻辑是否正确的可靠高效的方式。不过由于一些复杂逻辑和依赖库，会使得测试变得困难。一个使用Python mock 对象的理由就是在测试过程中控制代码的行为。比如代码发送HTTP请求到外部服务，只有当服务的行为符合您的预期时，您的测试才会可预测地执行。有时，这些外部服务行为的临时更改可能导致测试套件中的间歇性故障。因此，我们想要使我们的代码在一个受控的环境下测试。而使用mock对象可以做到这一点。

有时，很难测试代码的某些环节，比如except代码块，if代码块，因为可能不出现这样的场景，这时候使用mock对象也可以帮助控制代码执行的路径来使程序能运行到这些地方，提升code coverage。

另一个原因是更好地理解如何使用代码的真实副本。一个python mock对象包含关于其用法的数据，您可以检查这些数据，比如：是否调用了某个方法，如何调用某个方法，多久一次调用某个方法。

此外，有时候我们会面临这样的情形，即我们想测试我们的代码，但是不想产生一些脏结果，比如：我们想要测试facebook的上传功能，但是并不想真的上传一个内容上去。再比如，写一个弹出一个CD drive的脚本，或者一个从/tmp文件夹清除缓存的服务，或者一个绑定到TCP端口的socket服务，这些在unittest下都会产生dirty结果。作为写代码的，更关心的是您的库成功地调用了系统函数来弹出CD，而不是每次运行测试时都还需要打开CD drive。保持单元测试的效率和性能意味着尽量避免运行自动化测试的缓慢代码。

还有，个人认为，在实际测试算法代码的过程中，后面函数会用到前面过程的数据结果，如果每次都从头测试，那么花费时间会很长，因此保存中间计算结果，然后使用mock来代替前面的函数过程，直接读取中间结果来供后面代码测试也是十分必要的。

而unittest.mock可以客服这些困难。接下来就看看mock究竟是什么。

#### What Is Mocking?

mock就是“看起来像真的”的意思，在**测试环境下**，一个mock对象**代替模拟**一个真实的对象。是一个灵活有力的提升测试质量的工具。

unittest.mock库提供了一个叫做Mock的类，可以使用它来模拟代码中的真实对象。Mock还提供了一个函数patch()，它用Mock实例提到了代码中的真实对象。可以将patch()用为decorator，也可以用作context manager ， 取决于想要模拟的对象控制的scope。一旦退出指定的scope，patch就会立刻用真实的副本来取代mock对象。

首先先看看Mock。

In [108]:
from unittest.mock import Mock
mock = Mock()
mock

<Mock id='2262709476464'>

现在就可以使用 Mock来替代代码中的对象了。可以传递它为一个函数的参数或者重定义一个对象。形如：

```python
# Pass mock as an argument to do_something()
do_something(mock)

# Patch the json library
json = mock
```

注意，当你替换一个对象时，Mock必须要看起来真的像这个对象。比如要mock json库，那么程序调用dumps函数，你的mock对象里必须得有一个dumps函数。

In [109]:
mock.some_attribute

<Mock name='mock.some_attribute' id='2262709476560'>

In [110]:
mock.do_something()

<Mock name='mock.do_something()' id='2262709930496'>

Mock可以创建任意属性，可以代替任意对象。用一下之前提到的json例子：

In [111]:
json = Mock()
json.dumps()

<Mock name='mock.dumps()' id='2262709931408'>

可以看到很容易的就mock了kson库和其dumps函数，dumps可以接受任意参数，返回值也是一个mock对象，因此mock可以用到很复杂的环境下。很灵活。

接下来，看看如何用mock更好地理解代码。Mock实例存储这怎么使用它们的数据。

首先可以断言程序使用了你期望的一个对象。

In [112]:
from unittest.mock import Mock
json = Mock()
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='2262710018352'>

In [113]:
json.loads.assert_called()

In [114]:
json.loads.assert_called_with('{"key": "value"}')

In [115]:
json.loads.assert_called_once_with('{"key": "value"}')

In [116]:
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='2262710018352'>

In [117]:
json.loads.assert_called_once()

AssertionError: Expected 'loads' to have been called once. Called 2 times.
Calls: [call('{"key": "value"}'), call('{"key": "value"}')].

In [118]:
json.loads.assert_called_once_with('{"key": "value"}')

AssertionError: Expected 'loads' to be called once. Called 2 times.
Calls: [call('{"key": "value"}'), call('{"key": "value"}')].

In [119]:
json.loads.assert_not_called()

AssertionError: Expected 'loads' to not have been called. Called 2 times.
Calls: [call('{"key": "value"}'), call('{"key": "value"}')].

.assert_called()函数确保了调用mocked函数。 .assert_called_once()可以检查调用的次数。

第二，可以查看特殊属性以理解应用是如何使用该对象的。

In [120]:
from unittest.mock import Mock
json = Mock()
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='2262710019744'>

In [121]:
json.loads.call_count

1

In [122]:
json.loads.call_args

call('{"key": "value"}')

In [123]:
json.loads.call_args_list

[call('{"key": "value"}')]

In [124]:
json.method_calls

[call.loads('{"key": "value"}')]

通过以上测试代码可以使用各类属性来保证对象行为是想要的。这是一些固有的方法，接下来看看如何定制mocked方法。

管理一个Mock的返回值。一个使用mocks的原因就是控制代码的行为。一种十分常用的方式就是指定一个函数的返回值。

首先，创建一个文件my_calendar.py，代码见文件。然后执行下列语句：

In [125]:
!python my_calendar.py

上述代码如果在周末的时候运行是会报错的。而平常是正确的。写测试代码时候，很重要的是确保结果是可预测的。可以使用Mock来去除代码中的不确定性。如下所示，通过Mock .today() 指定返回值来实现。

In [126]:
import datetime
from unittest.mock import Mock

# Save a couple of test days
tuesday = datetime.datetime(year=2019, month=1, day=1)
saturday = datetime.datetime(year=2019, month=1, day=5)

# Mock datetime to control today's date
datetime = Mock()

def is_weekday():
    today = datetime.datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return (0 <= today.weekday() < 5)

# Mock .today() to return Tuesday
datetime.datetime.today.return_value = tuesday
# Test Tuesday is a weekday
assert is_weekday()
# Mock .today() to return Saturday
datetime.datetime.today.return_value = saturday
# Test Saturday is not a weekday
assert not is_weekday()

#### patch()

前面已经提到，unittest.mock还有一个很好的机制：patch(), 装饰器，补丁。

接下来通过实例分析。

看下mock官方的说明：“mock is a library for testing in Python. It allows you to replace parts of your system under test with mock objects and make assertions about how they have been used.”

关键是如何理解“replace parts of your system”。这里的parts指的是什么。实际上可以包括：

- functions
- classes，objects

可以使用Mock 或 MagicMock 类的实例的“mock 对象”来替代它们。

比如mocking functions，以一个简单的函数为例，文件 simple.py。在测试中调用该函数有两种方式，其一是直接使用：

In [127]:
import simple
from unittest import mock
def use_simple_function():
    result = simple.simple_function()
    print(result)
use_simple_function()

You have called simple_function


其二就是利用mock进行测试，为了模仿 simple_function，可以使用 mock.patch decorator。该decorator可以使用户通过以‘package.module.FunctionName’形式传入字符串参数指定想要mock的内容。对于本例，即module simple和函数simple_function，decorator如下第一行，函数可以表达为如下所示。其中函数mock_simple_function的参数是MagicMock类对象，用来代替想要mock的函数。

In [128]:
@mock.patch('simple.simple_function')
def mock_simple_function(mock_simple_func):
    print(mock_simple_func)
mock_simple_function()

<MagicMock name='simple_function' id='2262709874752'>


通过语句“@mock.patch(‘simple.simple_function’)”表明想使用MagicMock对象表达来替代simple_function，这个对象放入了mock_simple_func这一函数形参中。如上代码执行结果所示，输出是一个MagicMock对象，就是它替代了simple_function被调用。可从以下代码中看出：

In [129]:
@mock.patch('simple.simple_function')
def mock_simple_function(mock_simple_func):
    print(mock_simple_func)
    print(simple.simple_function)
    result = simple.simple_function()
    print(result)
mock_simple_function()

<MagicMock name='simple_function' id='2262710350848'>
<MagicMock name='simple_function' id='2262710350848'>
<MagicMock name='simple_function()' id='2262709875712'>


不过现在还有个更重要的问题：为什么创建新的MagicMock对象，究竟怎么调用这个对象。所以要看看MagicMock这个类。MagicMock之所以叫magic，是因为它有大多数python的magic函数的默认实现，即那些名称前后有双下划线的函数，可以查看[这里](http://www.ironpythoninaction.com/magic-methods.html)。比如__call__，即可以让一个类对象可以像函数那样被调用。因此，MagicMock对象是可以直接像函数那样被调用的。

回到例子中，现在已经mock了simple_function，但是还没有使用它来做什么。那么现在想返回simple_function()的结果要怎么做呢？可以使用MagicMock的return_value属性来实现：

In [130]:
@mock.patch('simple.simple_function')
def mock_simple_function(mock_simple_func):
    mock_simple_func.return_value = "You have mocked simple_function"
    result = simple.simple_function()
    print(result)
mock_simple_function()

You have mocked simple_function


从上面结果可以看出很好地模仿了simple_function函数结果。

如果除了返回值之外，还想其他功能，可以使用MagicMock.side_effect 。比如想要测试一个错误并抛出一个异常。

In [131]:
def side_effect_function():
    raise FloatingPointError("A disastrous floating point error has occurred")

@mock.patch('simple.simple_function')
def mock_simple_function_with_side_effect(mock_simple_func):
    mock_simple_func.side_effect = side_effect_function
    result = simple.simple_function()
    print(result)
    
mock_simple_function_with_side_effect()

FloatingPointError: A disastrous floating point error has occurred

接下来看一看如何mock类。在simple.py 文件中定义一个类。然后定义一个调用的函数，首先还是传统的调用方式：

In [132]:
import simple
def use_simple_class():
    inst = simple.SimpleClass()
    print(inst.explode())
use_simple_class()

KABOOM!


然后接下来看看mock下如何操作。依然使用@mock.patch decorator

In [133]:
from unittest import mock
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
mock_simple_class()

<MagicMock name='SimpleClass' id='2262710318080'>


通过@mock.patch decorator ，参数mock_class使用了MagicMock对象来代替了SimpleClass对象。

In [134]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
    print(simple.SimpleClass)
mock_simple_class()

<MagicMock name='SimpleClass' id='2262710360816'>
<MagicMock name='SimpleClass' id='2262710360816'>


接下来创建一个SimpleClass实例，然后打印，看看会发生什么。

In [135]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
    print(simple.SimpleClass)
    inst = simple.SimpleClass()
    print(inst)
mock_simple_class()

<MagicMock name='SimpleClass' id='2262708013568'>
<MagicMock name='SimpleClass' id='2262708013568'>
<MagicMock name='SimpleClass()' id='2262710383232'>


可以看出，调用SimpleClass() 就调用了MagicMock对象作为函数来创建了MagicMock对象。

到这里，mock一个函数和mock一个类并没有什么区别。不过在类中，使用的更多是其对象。从下面的例子中可以看出，类的MagicMock对象返回值是类对象。

简单小结一下，就是mock一个class时创建了一个MagicMock对象。创建一个类对象时，新的MagicMock对象也被创建，另外类的MagicMock对象返回值也就是类对象的MagicMock对象。

In [136]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
    print(simple.SimpleClass)
    inst = simple.SimpleClass()
    print(inst)
    print(mock_class.return_value)
mock_simple_class()

<MagicMock name='SimpleClass' id='2262710354896'>
<MagicMock name='SimpleClass' id='2262710354896'>
<MagicMock name='SimpleClass()' id='2262710465152'>
<MagicMock name='SimpleClass()' id='2262710465152'>


此外，可以在类对象中通过explode函数来设置return_value，以mock 类对象的返回值。

In [137]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    mock_class.return_value.explode.return_value = "BOO!"
    inst = simple.SimpleClass()
    result = inst.explode()
    print(result)
    print(mock_class.return_value)
mock_simple_class()

BOO!
<MagicMock name='SimpleClass()' id='2262685234320'>


### 给程序性能测试

测试程序运行所花费的时间并做性能测试。如果只是简单的想测试下程序整体花费的时间， 通常使用Unix时间函数就行了，比如：

```code
bash % time python3 someprogram.py
real 0m13.937s
user 0m12.162s
sys  0m0.098s
bash %
```

如果你还需要一个程序各个细节的详细报告，可以使用 cProfile 模块：

```code
bash % python3 -m cProfile someprogram.py
bash %
```

不过通常情况是介于这两个极端之间。比如已经知道代码运行时在少数几个函数中花费了绝大部分时间。 对于这些函数的性能测试，可以使用一个简单的装饰器。要使用这个装饰器，只需要将其放置在要进行性能测试的函数定义前即可。

In [138]:
# timethis.py

import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

@timethis
def countdown(n):
     while n > 0:
            n -= 1

countdown(10000000)

__main__.countdown : 1.0297941000000037


对于测试很小的代码片段运行性能，使用 timeit 模块会很方便

In [139]:
from timeit import timeit
timeit('math.sqrt(2)', 'import math')
timeit('sqrt(2)', 'from math import sqrt')
timeit('math.sqrt(2)', 'import math', number=10000000)
timeit('sqrt(2)', 'from math import sqrt', number=10000000)

1.3171404999999936

当执行性能测试的时候，需要注意的是你获取的结果都是近似值。 time.perf_counter() 函数会在给定平台上获取最高精度的计时值。 不过，它仍然还是基于时钟时间，很多因素会影响到它的精确度，比如机器负载。 如果你对于执行时间更感兴趣，使用 time.process_time() 来代替它。

In [140]:
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

### 加速程序运行

程序运行太慢，想在不使用复杂技术比如C扩展或JIT编译器的情况下加快程序运行速度。

关于程序优化的第一个准则是“不要优化”，第二个准则是“不要优化那些无关紧要的部分”。 如果你的程序运行缓慢，首先得对它进行性能测试找到问题所在。

通常来讲会发现程序在少数几个热点地方花费了大量时间， 比如内存的数据处理循环。一旦定位到这些点，就可以使用下面这些实用技术来加速程序运行。

#### 使用函数

很多程序员刚开始会使用Python语言写一些简单脚本。 当编写脚本的时候，通常习惯了写毫无结构的代码。比如：

```python
# somescript.py

import sys
import csv

with open(sys.argv[1]) as f:
     for row in csv.reader(f):

         # Some kind of processing
         pass
```

像这样定义在全局范围的代码运行起来要比定义在函数中运行慢的多。 这种速度差异是由于局部变量和全局变量的实现方式（**使用局部变量要更快些**）。 因此，如果想让程序运行更快些，只需要将脚本语句放入函数中即可：

```python
# somescript.py
import sys
import csv

def main(filename):
    with open(filename) as f:
         for row in csv.reader(f):
             # Some kind of processing
             pass

main(sys.argv[1])
```

根据经验，使用函数带来15-30%的性能提升是很常见的。

局部变量会比全局变量运行速度快。 对于频繁访问的名称，通过将这些名称变成局部变量可以加速程序运行。

对于类中的属性访问也同样适用于这个原理。 通常来讲，查找某个值比如 self.name 会比访问一个局部变量要慢一些。 在内部循环中，可以将某个需要频繁访问的属性放入到一个局部变量中。

#### 尽可能去掉属性访问

每一次**使用点(.)操作符来访问属性的时候会带来额外的开销**。 它会触发特定的方法，比如 __getattribute__() 和 __getattr__() ，这些方法会进行字典操作操作。

通常你可以使用 from module import name 这样的导入形式，以及使用绑定的方法。比如下面的函数是耗时的：

```python
import math

def compute_roots(nums):
    result = []
    for n in nums:
        result.append(math.sqrt(n))
    return result

# Test
nums = range(1000000)
for n in range(100):
    r = compute_roots(nums)
```

可以修改compute_roots函数如下：

In [141]:
from math import sqrt

def compute_roots(nums):

    result = []
    result_append = result.append
    for n in nums:
        result_append(sqrt(n))
    return result

修改后的版本运行时间会减少一些。唯一不同之处就是消除了属性访问。 用 sqrt() 代替了 math.sqrt() 。 The result.append() 方法被赋给一个局部变量 result_append ，然后在内部循环中使用它。

这些改变只有在大量重复代码中才有意义，比如循环。 因此，这些优化也只是在某些特定地方才应该被使用。

#### 避免不必要的抽象

任何时候当你使用额外的处理层（比如装饰器、属性访问、描述器）去包装你的代码时，都会让程序运行变慢。

In [142]:
class A:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    @property
    def y(self):
        return self._y
    @y.setter
    def y(self, value):
        self._y = value
        
from timeit import timeit
a = A(1,2)
timeit('a.x', 'from __main__ import a')

0.044568299999980354

In [143]:
timeit('a.y', 'from __main__ import a')

0.14513039999999933

访问属性y相比属性x而言慢的不止一点点，大概慢了4.5倍。 如果你在意性能的话，那么就需要重新审视下对于y的属性访问器的定义是否真的有必要了。 如果没有必要，就使用简单属性吧。 如果仅仅是因为其他编程语言需要使用getter/setter函数就去修改代码风格，这个真的没有必要。

#### 使用内置的容器

内置的数据类型比如字符串、元组、列表、集合和字典都是使用C来实现的，运行起来非常快。 如果想自己实现新的数据结构（比如链接列表、平衡树等）， 那么要想在性能上达到内置的速度几乎不可能，因此，还是乖乖的使用内置的吧。

另外，还要避免创建不必要的数据结构或复制。

#### 并行编程

这部分有参考：[Python性能优化的20条建议](https://segmentfault.com/a/1190000000666603)。

可以通过内置的模块multiprocessing实现下面几种并行模式：

多进程：对于CPU密集型的程序，可以使用multiprocessing的Process,Pool等封装好的类，通过多进程的方式实现并行计算。但是因为进程中的通信成本比较大，对于进程之间需要大量数据交互的程序效率未必有大的提高。

多线程：对于IO密集型的程序，multiprocessing.dummy模块使用multiprocessing的接口封装threading，使得多线程编程也变得非常轻松(比如可以使用Pool的map接口，简洁高效)。

分布式：multiprocessing中的Managers类提供了可以在不同进程之共享数据的方式，可以在此基础上开发出分布式的程序。

不同的业务场景可以选择其中的一种或几种的组合实现程序性能的优化。

#### 讨论

**在优化之前，有必要先研究下使用的算法**。 选择一个复杂度为 O(n log n) 的算法要比你去调整一个复杂度为 O(n**2) 的算法所带来的性能提升要大得多。

如果你觉得你还是得进行优化，那么请从整体考虑。 作为一般准则，不要对程序的每一个部分都去优化,因为这些修改会导致代码难以阅读和理解。 你应该**专注于优化产生性能瓶颈的地方，比如内部循环**。

对循环的优化所遵循的原则是尽量减少循环过程中的计算量，有多重循环的尽量将内层的计算提到上一层———[Python 代码性能优化技巧](https://www.ibm.com/developerworks/cn/linux/l-cn-python-optim/index.html)。

这里对循环做些补充，参考：[Python性能诀窍](http://pfmiles.github.io/blog/python-speed-performance-tips/).

Python支持好几种循环结构。for语句是最常用的。它遍历一个序列的每个元素，将每个元素赋值给循环变量。如果你的循环体很简单，for循环本身的解释成本将占据大部分的开销。这个时候**map函数**就能派上用场了。你可以将map函数看作是for循环采用C代码来实现。唯一的约束是“**循环体”必须是一个函数调用**。**list comprehension 列表生成式**除了语法上的便利性之外，他们常常和等价的map调用一样快甚至更快。比如：

```python
newlist = []
for word in oldlist:
    newlist.append(word.upper())
```

可以使用map函数将这个循环由解释执行推到编译好的C代码中去执行：

```python
newlist = map(str.upper, oldlist)
```

List comprehension在python 2.0的时候被加入。它们提供了一种更紧凑的语法和更高效的方式来表达上面的for循环：

```python
newlist = [s.upper() for s in oldlist]
```

如果优化要求比较高，本节的这些简单技术满足不了，那么可以研究下基于即时编译（JIT）技术的一些工具。 例如，PyPy工程是Python解释器的另外一种实现，它会分析程序运行并对那些频繁执行的部分生成本机机器码。 它有时候能极大的提升性能，通常可以接近C代码的速度。 不过可惜的是，PyPy还不能完全支持Python3.。

还可以考虑下Numba工程， Numba是一个在你使用装饰器来选择Python函数进行优化时的动态编译器。 这些函数会使用LLVM被编译成本地机器码。它同样可以极大的提升性能。 但是，跟PyPy一样，它对于Python 3的支持现在还停留在实验阶段。

最后引用John Ousterhout说过的话作为结尾：“最好的性能优化是从不工作到工作状态的迁移”。 直到你真的需要优化的时候再去考虑它。确保你程序正确的运行通常比让它运行更快要更重要一些（至少开始是这样的）.